In this competition the baseline predictions are provided for object detection task for both train and test set. You may wonder how they are accurate, here I'll test it!

I'll use coco mAP score since it's the most common way to evaluate object detection model.

I've already converted ground truth annotation to coco format here.  
https://www.kaggle.com/bamps53/create-coco-format-annotations

The converted json file is also provided.  
https://www.kaggle.com/bamps53/nfl2021traincocoannotations

In this notebook, I'll convert `train_baseline_helmets.csv` to coco prediction format.
And then evaluate it with pycocotools.

In [ ]:
!pip install pycocotools

In [ ]:
import numpy as np
import pandas as pd
import json
from tqdm.auto import tqdm

def get_file_name(video_frame):
    video_frame = video_frame.split('_')
    video = "_".join(video_frame[:-1])
    frame = int(video_frame[-1])
    file_name = f'{video}_frame{frame:04}.jpg'
    return file_name

train_annotations = json.load(open('../input/nfl2021traincocoannotations/train.json'))

train_preds = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/train_baseline_helmets.csv')
train_preds['file_name'] = train_preds['video_frame'].map(get_file_name)

images = pd.DataFrame(train_annotations['images'])
train_preds = train_preds.merge(images[['file_name', 'id']])

In [ ]:
# convert to coco prediction format and save it as json.
predictions = []
for _, row in tqdm(train_preds.iterrows(), total=len(train_preds)):
    pred = {
        "image_id": row['id'],
        "category_id": 1,
        "bbox": [row['left'], row['top'], row['height'], row['width']],
        "score": row['conf'],
        }
    predictions.append(pred)
    
json.dump(predictions, open('results.json', 'w'))

In [ ]:
# evaluate with pycocotools
import pycocotools.coco as coco
from pycocotools.cocoeval import COCOeval

coco_ann = coco.COCO('../input/nfl2021traincocoannotations/train.json')
coco_dets = coco_ann.loadRes('results.json')
coco_eval = COCOeval(coco_ann, coco_dets, "bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()